# 29. Integrando LangChain com n8n (Webhook Client)

O **n8n** é uma ferramenta poderosa para conectar serviços (Slack, Google Sheets, Email, etc). Ao invés de reimplementar toda essa lógica em Python, podemos usar o LangChain apenas como o "Cérebro" e o n8n como os "Músculos".

**Objetivos:**
1. Entender como acionar workflows do n8n via Webhook (HTTP POST).
2. Criar uma **Tool** do LangChain que encapsula essa chamada.
3. Criar um Agente que decide *quando* chamar o n8n.

---

In [ ]:
!pip install -q langchain langchain-openai

In [ ]:
import os
import getpass
import requests
import json

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")

## 1. O Conceito de Webhook

No n8n, você cria um workflow que começa com um nó **Webhook**. Ele te dá uma URL (ex: `https://seu-n8n.com/webhook/test`).

Quando fazemos um POST para essa URL enviando JSON, o n8n recebe os dados e inicia o processo.

In [ ]:
# URL de Exemplo (Substitua pela sua URL real do n8n)
# Se não tiver um n8n rodando, usaremos um mock para teste
N8N_WEBHOOK_URL = "https://postman-echo.com/post" # Mock que retorna o que enviamos

def trigger_n8n_workflow(data: dict):
    """Envia dados para o n8n."""
    try:
        response = requests.post(N8N_WEBHOOK_URL, json=data)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        return f"Erro ao chamar n8n: {e}"

# Teste manual
print(trigger_n8n_workflow({"message": "Olá n8n!", "source": "Colab"}))

## 2. Encapsulando como Ferramenta LangChain

Agora vamos dar poder ao Agente. Imagine que temos um workflow no n8n que **Adiciona uma Linha no Google Sheets**.

In [ ]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

# 1. Definir Schema dos dados que o n8n espera para esse workflow específico
class AddSpreadsheetRowInput(BaseModel):
    nome: str = Field(description="Nome do cliente")
    email: str = Field(description="Email do cliente")
    interesse: str = Field(description="Resumo do interesse do cliente")

# 2. Função da Tool
def add_to_spreadsheet(nome: str, email: str, interesse: str) -> str:
    """Envia os dados para o n8n adicionar na planilha de Leads."""
    payload = {
        "action": "add_row",
        "data": {
            "nome": nome,
            "email": email,
            "interesse": interesse
        }
    }
    # Chamada real (simulada aqui)
    result = trigger_n8n_workflow(payload)
    return f"Comando enviado ao n8n. Resposta do servidor: {result}"

# 3. Criar a Tool
n8n_tool = StructuredTool.from_function(
    func=add_to_spreadsheet,
    name="AddLeadToSheets",
    description="Use esta ferramenta SEMPRE que precisar salvar um novo lead ou cliente na planilha.",
    args_schema=AddSpreadsheetRowInput
)

## 3. Agente em Ação

O Agente recebe um texto não estruturado (ex: email ou conversa) e decide estruturar e chamar o n8n.

In [ ]:
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain import hub
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, [n8n_tool], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[n8n_tool], verbose=True)

input_text = """
Recebi um contato de um tal de Carlos Silva.
O email dele é carlos.silva@exemplo.com e ele disse que queria comprar um seguro de vida.
"""

agent_executor.invoke({"input": f"Processe esse texto e salve se for relevante: {input_text}"})